In [1]:
import pandas as pd

In [2]:
paths = []

for depth in range(0, 100, 5):
    paths.append(f"/fsx/phuc/projects/nanotron/examples/infinite-context-length/scripts/needle_finetune_format_dataset_but_for_evals_32768_ctx_and_{depth}_depth.pkl")

In [3]:
dfs = [pd.read_pickle(path) for path in paths]
df = pd.concat(dfs, )  # Set ignore_index to reset index

In [4]:
df.head()

,id,prompt,answer,context_length,depth_percent,response
0,1ac90da2-3ff2-4bdd-98fe-5408627eacb0,There is an important info hidden inside a lot...,49885,32768,0.0,12.5. The pass key is 12.5. The pass key is 12...
1,aa95b758-d515-46d2-9c7f-1cc39f957fb3,There is an important info hidden inside a lot...,10520,32768,0.0,123. The pass key is 123. The pass key is 123....
2,1410ffa8-836e-4272-ade7-02e124965ded,There is an important info hidden inside a lot...,48327,32768,0.0,123. The pass key is 123. The pass key is 123....
3,bdfc0228-3464-43ed-bcab-d745e277b730,There is an important info hidden inside a lot...,45312,32768,0.0,1000. The pass key is 1000. The pass key is 10...
4,1eb5c055-aeab-4ff6-9462-0d4b10b9c558,There is an important info hidden inside a lot...,34160,32768,0.0,1000. The pass key is 1000. The pass key is 10...


In [5]:
df['depth_percent'].value_counts()

depth_percent
0.0     100
5.0     100
90.0    100
85.0    100
80.0    100
75.0    100
70.0    100
65.0    100
60.0    100
55.0    100
50.0    100
45.0    100
40.0    100
35.0    100
30.0    100
25.0    100
20.0    100
15.0    100
10.0    100
95.0    100
Name: count, dtype: int64

In [6]:
NEEDLE_TEXT = "answer"
DEPTH_TEXT = "depth_percent" # depth_percent

df["match"] = df.apply(lambda x: int(str(x[NEEDLE_TEXT]) in x["response"]), axis=1)
df_result = df[[DEPTH_TEXT, "context_length", "match"]].groupby(by=[DEPTH_TEXT, "context_length"]).mean().unstack(level="context_length")
df_result.columns = df_result.columns.droplevel(level=0)

In [7]:
# 95, and 100 works near perfect

df_result

context_length,32768
depth_percent,
0.0,0.00
5.0,0.00
10.0,0.00
15.0,0.00
20.0,0.00
25.0,0.00
30.0,0.00
35.0,0.00
40.0,0.00


In [ ]:
import re
def calculate_std(x, NEEDLE_TEXT):
    try:
        expected_needle = float(str(x[NEEDLE_TEXT]))
        needle_in_response = float(re.search(r"[-+]?\d*\.\d+|\d+", x["response"]).group())
        diff = abs(expected_needle - needle_in_response)
        if diff == 0:
            return 0
        elif diff <= 10:
            return 10
        elif diff <= 20:
            return 20
        elif diff <= 50:
            return 50
        else:
            return 100
    except (AttributeError, ValueError):
        return None

df["std"] = df.apply(lambda x: calculate_std(x, NEEDLE_TEXT), axis=1)
df_result = df[[DEPTH_TEXT, "context_length", "std"]].groupby(by=[DEPTH_TEXT, "context_length", "std"]).size().unstack(level="std")
df_result = df_result.fillna(0).astype(int)
df_result["total"] = df_result.sum(axis=1)

In [ ]:
df_result

In [ ]:
def get_std_and_match(x, NEEDLE_TEXT):
    try:
        expected_needle = float(str(x[NEEDLE_TEXT]))
        needle_in_response = float(re.search(r"[-+]?\d*\.\d+|\d+", x["response"]).group())
        diff = abs(expected_needle - needle_in_response)
        if diff == 0:
            return 0, 1
        elif diff <= 10:
            return 10, 1
        elif diff <= 20:
            return 20, 1
        elif diff <= 50:
            return 50, 1
        else:
            return 100, 1
    except (AttributeError, ValueError):
        return None, 0

df[["std", "match"]] = df.apply(lambda x: pd.Series(get_std_and_match(x, NEEDLE_TEXT)), axis=1)
df_result = df[[DEPTH_TEXT, "context_length", "std", "match"]].groupby(by=[DEPTH_TEXT, "context_length", "std"]).mean().unstack(level="std")
df_result.columns = df_result.columns.droplevel(level=0)
df_result.columns.name = "std"

In [ ]:
df_result = df_result.apply(lambda x: x / x["total"], axis=1).drop("total", axis=1)
df_result.columns.name = "std"

In [ ]:
df[df['depth_percent'] == 65].head(n=5)

In [ ]:
pd.set_option('display.max_rows', None)  # None means show all rows
pd.set_option('display.max_columns', None)  # None means show all columns
pd.set_option('display.max_colwidth', None)  # None means no truncation


### Context Length

In [12]:
heatmap_data

array([[0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
        0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]])

In [15]:
import plotly.graph_objects as go
import numpy as np

# Create data
depth = np.linspace(0, 100, 21)  # 0% to 100% in 5% increments
context_length = [16384]  # 8192 * 2

# Create the heatmap data
heatmap_data = np.zeros((1, len(depth)))
# heatmap_data[:] = np.linspace(0, 1, len(depth))
heatmap_data[:, :11] = 0  # Red for 0% to 50%
heatmap_data[:, 11:] = 1  # Green for 55% to 100

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data,
    x=depth,
    y=context_length,
    colorscale=[
        [0, 'red'],
        [0.5, 'yellow'],
        [1, 'green']
    ],
    zmin=0,
    zmax=1,
    showscale=True,
))

# Update layout
fig.update_layout(
    title="[Exp 55: 8b llama3's continual pretraining] Context Length vs. Depth",
    xaxis_title='Depth (%)',
    yaxis_title='Context Length',
    xaxis=dict(
        tickmode='linear',
        tick0=0,
        dtick=5,
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=[16384],
        ticktext=['16384'],
    ),
    width=800,
    height=400,
)

# Show the plot
fig.show()